# load and preprocessing the dataset

In [2]:
import mne
import pandas as pd
import numpy as np
#data = load_ndsp_data('D:\\jupyter_folder\\nurun_python_codes\\pre_Sb.npy')
raw= mne.io.read_raw_eeglab('D:\WM_openneuro_dataset\sub-064\eeg\sub-064_task-rest_eeg.set')
all_events, all_event_id = mne.events_from_annotations(raw)
my_events= mne.pick_events(all_events)
epochs = mne.Epochs(raw,my_events, tmin=0.0, tmax=240.0, baseline=(0, 0))
data=epochs.to_data_frame()
data.drop('time', axis=1, inplace=True)
data.drop('condition', axis=1, inplace=True)
data.drop('epoch', axis=1, inplace=True)
data.drop([0],axis=0,inplace=True)
######################################################################################################
edata=data.to_numpy(dtype ='float32')
clm,rw =data.shape
#eeg=np.empty([rw,clm])dtype ='float32'
eeg=np.reshape(edata,(rw,-1))
######################################################################################################
from meegkit.asr import ASR
from meegkit.utils.matrix import sliding_window
from scipy.signal import butter, lfilter


def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def apply_asr(raw, sfreq=1000, cutoff=2.5, blocksize=100, win_len=0.5,
        win_overlap=0.66, max_dropout_fraction=0.1,
        min_clean_fraction=0.25, name='asrfilter', method='euclid',
        estimator='scm'):
   
    h, w = raw.shape
    
    # t = int(raw.shape[1] / sfreq)
    asr = ASR(sfreq=sfreq, cutoff=cutoff, blocksize=blocksize, win_len=win_len,
        win_overlap=win_overlap, max_dropout_fraction=max_dropout_fraction,
        min_clean_fraction=min_clean_fraction, name=name, method=method,
        estimator=estimator)
        
        # method='euclid'
    # train_idx = np.arange(0 * sfreq, t * sfreq, dtype=int)
    _, sample_mask = asr.fit(raw)
    
    # Apply filter using sliding (non-overlapping) windows
    X = sliding_window(raw, window=int(sfreq), step=int(sfreq))
    Y = np.zeros_like(X)
    for i in range(X.shape[1]):
        Y[:, i, :] = asr.transform(X[:, i, :])
    
    raw = X.reshape(h, -1)  # reshape to (n_chans, n_times)
    clean = Y.reshape(h, -1)
    return raw, clean

import pyedflib

filter_signal = butter_bandpass_filter(eeg, 0.5, 30, 1000)
signal, filter_asr_signal = apply_asr(filter_signal)
#np.save('s64_denoise_resting_state', filter_asr_signal)

C:\ProgramData\Anaconda3\lib\site-packages\mne\externals\pymatreader\utils.py:118: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
<ipython-input-2-734ddd8e2f47>:5: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw= mne.io.read_raw_eeglab('D:\WM_openneuro_dataset\sub-064\eeg\sub-064_task-rest_eeg.set')


Used Annotations descriptions: ['boundary', 'eyes opened']
Not setting metadata
Not setting metadata
2 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 2 events and 240001 original time points ...


<ipython-input-2-734ddd8e2f47>:5: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw= mne.io.read_raw_eeglab('D:\WM_openneuro_dataset\sub-064\eeg\sub-064_task-rest_eeg.set')
<ipython-input-2-734ddd8e2f47>:5: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw= mne.io.read_raw_eeglab('D:\WM_openneuro_dataset\sub-064\eeg\sub-064_task-rest_eeg.set')


1 bad epochs dropped


# absolout and relative band power

In [3]:
def bandpower(data, sf, band, method='welch', window_sec=None, relative=False):
   
    from scipy.signal import welch
    from scipy.integrate import simps
    from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * sf
        else:
            nperseg = (2 / low) * sf

        freqs, psd = welch(data, sf, nperseg=nperseg)

    elif method == 'multitaper':
        psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                          normalization='full', verbose=0)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp
fs=1000
from neurodsp.utils.download import load_ndsp_data
# Multitaper delta power
bp = bandpower(filter_asr_signal[1,:], fs, [0.5, 4], 'multitaper')
bp_rel = bandpower(filter_asr_signal[1,:], fs, [0.5, 4], 'multitaper', relative=True)
print('Absolute delta power: %.3f' % bp)
print('Relative delta power: %.3f' % bp_rel)

# Delta-beta ratio
# One advantage of the multitaper is that we don't need to define a window length.
db = bandpower(filter_asr_signal[1,:], fs, [0.5, 4], 'multitaper') / bandpower(filter_asr_signal[1,:], fs, [12, 30], 'multitaper')
print('Delta/beta ratio (absolute): %.3f' % db)

<ipython-input-3-05857fdf4620>:20: RuntimeWarning: Could not properly use low_bias, keeping lowest-bias taper
  psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
<ipython-input-3-05857fdf4620>:20: RuntimeWarning: Not adaptively combining the spectral estimators due to a low number of tapers (1 < 3).
  psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
<ipython-input-3-05857fdf4620>:20: RuntimeWarning: Could not properly use low_bias, keeping lowest-bias taper
  psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
<ipython-input-3-05857fdf4620>:20: RuntimeWarning: Not adaptively combining the spectral estimators due to a low number of tapers (1 < 3).
  psd, freqs = psd_array_multitaper(data, sf, adaptive=True,


Absolute delta power: 78.799
Relative delta power: 0.384


<ipython-input-3-05857fdf4620>:20: RuntimeWarning: Could not properly use low_bias, keeping lowest-bias taper
  psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
<ipython-input-3-05857fdf4620>:20: RuntimeWarning: Not adaptively combining the spectral estimators due to a low number of tapers (1 < 3).
  psd, freqs = psd_array_multitaper(data, sf, adaptive=True,


Delta/beta ratio (absolute): 1.390


<ipython-input-3-05857fdf4620>:20: RuntimeWarning: Could not properly use low_bias, keeping lowest-bias taper
  psd, freqs = psd_array_multitaper(data, sf, adaptive=True,
<ipython-input-3-05857fdf4620>:20: RuntimeWarning: Not adaptively combining the spectral estimators due to a low number of tapers (1 < 3).
  psd, freqs = psd_array_multitaper(data, sf, adaptive=True,


# amplitude asymmetry

In [3]:
Amplitude_assymetry=np.zeros((21,21))
for jj in range (0,20):
    for j in range (0,20):
        ch_m=filter_asr_signal[jj,:]
        ch_n=filter_asr_signal[j,:] 
        power_f=np.log(np.mean(np.abs(ch_m**2)))
        power_ff=np.log(np.mean(np.abs(ch_n**2)))
        Amplitude_assymetry[jj,j]=power_f-power_ff

# coherence

In [6]:
import scipy.io
import scipy.signal as signal
coherence1=np.zeros((21,21))
fs=1000
for jj in range (0,20):
    for j in range (0,20):
        ch_m=filter_asr_signal[jj,:]
        ch_n=filter_asr_signal[j,:] 
        power_f=np.log(np.mean(np.abs(ch_m**2)))
        power_ff=np.log(np.mean(np.abs(ch_n**2)))
        cx=signal.coherence(ch_m, ch_n, fs, window='hann', nperseg=256, noverlap=None, nfft=None, detrend='constant', axis=-1)

In [23]:
gg,cx=signal.coherence(filter_asr_signal[1,:], filter_asr_signal[2,:], fs, window='hann', noverlap=None, nfft=None, detrend='constant', axis=-1)

In [25]:
print(cx.shape)

(129,)


# phase lag

In [27]:
#Phase Lag
from scipy.signal import hilbert
phase_diff=np.zeros((21,21))
for jj in range (0,20):
    for j in range (0,20):
        ch_m=filter_asr_signal[jj,:]
        ch_n=filter_asr_signal[j,:] 
        x1h = hilbert(ch_m)
        x2h = hilbert(ch_n)
        c = np.inner( x1h, np.conj(x2h) ) / np.sqrt( np.inner(x1h,np.conj(x1h)) * np.inner(x2h,np.conj(x2h)) )
        phase_diff[jj,j]= np.angle(c)

# DFA

In [31]:
print(dfas)

[0.78306462 0.85904042 0.67413346 0.73047038 0.78383044 0.79070018
 0.59716397 0.79159262 0.57323082 0.62729282 0.56351986 0.59934
 0.49160154 0.44593767 0.41406926 0.55034509 0.35557157 0.36822528
 0.34480842 0.33460472 0.42244151]
